<a href="https://colab.research.google.com/github/akshaypt7/amazon_sales_prediction/blob/main/my_project_01_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is part 2 of the project. Here we make improvements that we have learnt from part 1 of the project.

One of the mistakes we have done in the first part was that there was data leakage. 

**When the sales are made for each asins in a day, we are not considering the asins that were not sold that day. so the thing is we need to add ASINS even when there was no sale made for that. We can give the quantity as 0. **


How do we do this?

When we read data each month, we see what are the unique ASINs, then these asins should be added to each days, even for the days when these asins are not sold. 
but what other details can we give, what about promotions and other details, we can take the entire row not just the ASIN, but what if there are two rows with coupon and without coupon, if we can see which row is closer to that, that is with coupon or without coupon we can copy that coupon.

one another thing to note is we can even look at working this for the enitre dataframe at once, rather than going month by month. The reason is what if for one month the product was laucnhed but on second month there was no sales, even though that product was available. this means that for all those we wont even mention the asin, but on the third month we mention again.

- we can change the continous and categorical variables that we have defined.

One another mistake that we had done in part 1 was that we had only taken 679 data points as the validation set, we have used 10,500 as the train set. we should have atleast taken 3000 units as validation set.

This was the reason we had high training error and less validation error. This makes sense why we also had high OOB_error and less validation error, as oob_error is calculated with atleast 30% data.

Next thing we learnt is we need to understand what metric we have to use.

**We can use both classification and regression. When we use both then we need to understand what has to be used as the metric.**

Also if we are considering accuracy or even other metrics we have to create a baseline model which predicts the majority class for all the data points.

Learn more about metrics.

One another thing what we can note is, we can convert 1.4 to 2, if we are going with regression. Here we need to come up with something like np.where(). We can see what is the performance at different thresholds. 

We should also try to create the functions without copying and trying on a smaller set to know if it works.

In [ ]:
#hide
!pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
# from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [ ]:
import pandas as pd

In [ ]:
import os
files = os.listdir('/content/gdrive/MyDrive/my_project_01')


In [ ]:
dict_files = {}
for i,file in enumerate(files):
  try:
    # dict_files[i] = pd.read_csv( '/content/drive/MyDrive/my_project_01/' +str(file),low_memory=False)
    dict_files[i] = pd.read_csv( '/content/gdrive/MyDrive/my_project_01/' +str(file),low_memory=False)
  except:
    # dict_files[i] = pd.read_excel( '/content/drive/MyDrive/my_project_01/' +str(file))
    dict_files[i] = pd.read_excel( '/content/gdrive/MyDrive/my_project_01/' +str(file))


We have to take in Invoice Amount instead of Principal amount (since it helps to sepearate canceled and refund orders)

In [ ]:
df_dicts = {}

for i in range(len(dict_files)):
  df_dicts[i] = dict_files[i][['Order Date','Quantity','Asin','Invoice Amount','Item Promo Discount']]


Testing out with 1 dataframe

,Order Date,Quantity,Asin,Invoice Amount,Item Promo Discount
0,2020-12-28 15:41:42,1,B07TM3LRVB,-1135.00,0.00
1,2020-12-25 18:50:21,1,B07TM3LRVB,-1135.00,0.00
2,2020-12-28 23:01:59,1,B07TM3LRVB,-1135.00,0.00
3,2020-12-23 10:27:16,1,B07TM3LRVB,-1135.00,0.00
4,2020-12-18 11:41:24,1,B07TM3LRVB,-1135.00,0.00
...,...,...,...,...,...
926,2021-01-31 21:15:29,1,B07TM3LRVB,1078.25,-56.75
927,2021-01-31 22:33:34,1,B08332221J,1424.05,-74.95
928,2021-01-31 21:22:51,1,B0849NLNTQ,1599.00,0.00
929,2021-01-31 23:32:25,1,B07TM3LRVB,1078.25,-56.75


In [ ]:
df = df_dicts[1]
df

,Order Date,Quantity,Asin,Invoice Amount,Item Promo Discount
0,2021-02-01 17:28:51,1,B07TM3LRVB,1135.00,0.00
1,2021-02-04 09:18:42,1,B07TM3LRVB,1078.25,-56.75
2,2021-02-12 00:30:53,1,B07TM3LRVB,1078.25,-56.75
3,2021-02-13 10:30:28,1,B0849NLNTQ,1599.00,0.00
4,2021-02-13 12:35:00,1,B0849NLNTQ,1519.05,-79.95
5,2021-02-14 18:47:44,1,B07KYFHTGF,1895.25,-99.75
6,2021-02-14 21:49:20,1,B07TM3LRVB,1078.25,-56.75
7,2021-02-18 07:55:57,1,B0849NLNTQ,1599.00,0.00
8,2021-02-24 14:11:53,1,B07TM3LRVB,1078.25,-56.75
9,2021-01-24 16:47:48,1,B0849NLNTQ,-1599.00,0.00


In [ ]:
df_copy['Date'] = df['Order Date'].dt.date

In [ ]:
df_copy.drop(['Order Date'], inplace = True, axis = 1)

In [ ]:
df_copy

,Quantity,Asin,Invoice Amount,Item Promo Discount,Date
0,1,B07TM3LRVB,1135.00,0.00,2021-02-01
1,1,B07TM3LRVB,1078.25,-56.75,2021-02-04
2,1,B07TM3LRVB,1078.25,-56.75,2021-02-12
3,1,B0849NLNTQ,1599.00,0.00,2021-02-13
4,1,B0849NLNTQ,1519.05,-79.95,2021-02-13
5,1,B07KYFHTGF,1895.25,-99.75,2021-02-14
6,1,B07TM3LRVB,1078.25,-56.75,2021-02-14
7,1,B0849NLNTQ,1599.00,0.00,2021-02-18
8,1,B07TM3LRVB,1078.25,-56.75,2021-02-24
9,1,B0849NLNTQ,-1599.00,0.00,2021-01-24


In [ ]:
df_copy.groupby(['Date', 'Asin','Invoice Amount','Item Promo Discount'])['Quantity'].sum()

Date        Asin        Invoice Amount  Item Promo Discount
2021-01-24  B0849NLNTQ  -1599.00         0.00                  1
2021-01-28  B08332221J  -1499.00         0.00                  1
2021-02-01  B07TM3LRVB   1135.00         0.00                  1
2021-02-04  B07TM3LRVB   1078.25        -56.75                 1
2021-02-12  B07TM3LRVB   1078.25        -56.75                 1
2021-02-13  B0849NLNTQ   1519.05        -79.95                 1
                         1599.00         0.00                  1
2021-02-14  B07KYFHTGF   1895.25        -99.75                 1
            B07TM3LRVB   1078.25        -56.75                 1
2021-02-18  B0849NLNTQ   1599.00         0.00                  1
2021-02-24  B07TM3LRVB   1078.25        -56.75                 1
Name: Quantity, dtype: int64

In [ ]:
cond  = df_copy['Invoice Amount'] > 0

In [ ]:
df = df_copy.where(cond )

In [ ]:
df

,Quantity,Asin,Invoice Amount,Item Promo Discount,Date
0,1.0,B07TM3LRVB,1135.00,0.00,2021-02-01
1,1.0,B07TM3LRVB,1078.25,-56.75,2021-02-04
2,1.0,B07TM3LRVB,1078.25,-56.75,2021-02-12
3,1.0,B0849NLNTQ,1599.00,0.00,2021-02-13
4,1.0,B0849NLNTQ,1519.05,-79.95,2021-02-13
5,1.0,B07KYFHTGF,1895.25,-99.75,2021-02-14
6,1.0,B07TM3LRVB,1078.25,-56.75,2021-02-14
7,1.0,B0849NLNTQ,1599.00,0.00,2021-02-18
8,1.0,B07TM3LRVB,1078.25,-56.75,2021-02-24
9,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.dropna(how='all',inplace=True)

In [ ]:
df_1 = pd.DataFrame(df.groupby(['Date', 'Asin','Invoice Amount','Item Promo Discount'])['Quantity'].sum())

In [ ]:
df_1['Date'] = pd.to_datetime(df_1['Date'])

In [ ]:
df_1 = df_1.reset_index()

In [ ]:
df_1

,Date,Asin,Invoice Amount,Item Promo Discount,Quantity
0,2021-02-01,B07TM3LRVB,1135.00,0.00,1.0
1,2021-02-04,B07TM3LRVB,1078.25,-56.75,1.0
2,2021-02-12,B07TM3LRVB,1078.25,-56.75,1.0
3,2021-02-13,B0849NLNTQ,1519.05,-79.95,1.0
4,2021-02-13,B0849NLNTQ,1599.00,0.00,1.0
5,2021-02-14,B07KYFHTGF,1895.25,-99.75,1.0
6,2021-02-14,B07TM3LRVB,1078.25,-56.75,1.0
7,2021-02-18,B0849NLNTQ,1599.00,0.00,1.0
8,2021-02-24,B07TM3LRVB,1078.25,-56.75,1.0


In [ ]:
df_2 = df_1.set_index('Date')

In [ ]:
df_2.loc['2021-02-12']

Asin                   B07TM3LRVB
Invoice Amount            1078.25
Item Promo Discount        -56.75
Quantity                        1
Name: 2021-02-12 00:00:00, dtype: object

In [ ]:
df_2.loc['2021-02-12']['Asin']

'B07TM3LRVB'

##### Using a different dataframe

**Item promo can lead to data_leakage, we might have to remove it and then get the dates in which we had coupons and ads in amazon, then use that**

In [ ]:
df_new = df_dicts[2]
df_new_1 = df_new.copy()

In [ ]:
df_new_1['date'] = df_new['Order Date'].dt.date

In [ ]:
df_new  = df_new_1.drop(['Order Date'],axis=1)

In [ ]:
df_new

,Quantity,Asin,Invoice Amount,Item Promo Discount,date
0,1,B07TM3LRVB,-1135.00,0.00,2020-12-28
1,1,B07TM3LRVB,-1135.00,0.00,2020-12-25
2,1,B07TM3LRVB,-1135.00,0.00,2020-12-28
3,1,B07TM3LRVB,-1135.00,0.00,2020-12-23
4,1,B07TM3LRVB,-1135.00,0.00,2020-12-18
...,...,...,...,...,...
926,1,B07TM3LRVB,1078.25,-56.75,2021-01-31
927,1,B08332221J,1424.05,-74.95,2021-01-31
928,1,B0849NLNTQ,1599.00,0.00,2021-01-31
929,1,B07TM3LRVB,1078.25,-56.75,2021-01-31


In [ ]:
asin_list = df_new.Asin.unique()
asin_list

array(['B07TM3LRVB', 'B0849NLNTQ', 'B08332221J', 'B07MK32L49', 'B08976V1BZ', 'B0859193NJ', 'B07TJ1HDTM', 'B0858X26QM', 'B07KYFHTGF', 'B0859B5J8T', 'B0897LPBHR', 'B084JJWJ31', 'B07W1YYQ9G',
       'B07ZBCTB9N', 'B07S8NC532', 'B07YC69QKT', 'B07Z7QX6ST'], dtype=object)

In [ ]:
df_asin = pd.DataFrame()

In [ ]:
df_new.dtypes

Quantity                 int64
Asin                    object
Invoice Amount         float64
Item Promo Discount    float64
date                    object
dtype: object

In [ ]:
df_new['date'] = pd.to_datetime(df_new['date'])

In [ ]:

df_new.dtypes

Quantity                        int64
Asin                           object
Invoice Amount                float64
Item Promo Discount           float64
date                   datetime64[ns]
dtype: object

##### Removing refund and returns

In [ ]:
df_new.head()

,Quantity,Asin,Invoice Amount,Item Promo Discount,date
0,1,B07TM3LRVB,-1135.0,0.0,2020-12-28
1,1,B07TM3LRVB,-1135.0,0.0,2020-12-25
2,1,B07TM3LRVB,-1135.0,0.0,2020-12-28
3,1,B07TM3LRVB,-1135.0,0.0,2020-12-23
4,1,B07TM3LRVB,-1135.0,0.0,2020-12-18


In [ ]:
cond = df_new['Invoice Amount'] > 0

df_new_2  = df_new.where(cond)
df_new_2.dropna(how='all',inplace = True)

In [ ]:
df_new_2.head()


,Quantity,Asin,Invoice Amount,Item Promo Discount,date
22,1.0,B07TM3LRVB,1135.0,0.0,2020-12-31
23,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
24,1.0,B08332221J,1499.0,0.0,2021-01-01
25,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
26,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01


In [ ]:
df_new.shape

(931, 5)

In [ ]:
df_new_2.shape

(760, 5)

In [ ]:
max_date = max(df_new_2['date']) 
min_date = (min(df_new_2['date']))

In [ ]:
df_new_2.loc[df_new_2['date']== '2020-12-31 ']

,Quantity,Asin,Invoice Amount,Item Promo Discount,date
22,1.0,B07TM3LRVB,1135.0,0.0,2020-12-31


In [ ]:
# finding all the dates btw max and min

date_list  = pd.date_range(min_date,max_date)

In [ ]:
df_asin = pd.DataFrame(columns = ['date', 'Asin'])

In [ ]:
df_asin

,date,Asin


In [ ]:
index = 0
while True:

  for date in (date_list) :
    for asin in (asin_list):
      df_asin.loc[index] = [date,asin]
      index = index + 1

  break    

In [ ]:
df_1 = df_asin.head(5)
df_1

,date,Asin
0,2020-12-31,B07TM3LRVB
1,2020-12-31,B0849NLNTQ
2,2020-12-31,B08332221J
3,2020-12-31,B07MK32L49
4,2020-12-31,B08976V1BZ


In [ ]:
df_2 = df_new_2.head(5)
df_2

,Quantity,Asin,Invoice Amount,Item Promo Discount,date
22,1.0,B07TM3LRVB,1135.0,0.0,2020-12-31
23,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
24,1.0,B08332221J,1499.0,0.0,2021-01-01
25,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
26,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01


In [ ]:
df_1_index = df_1.set_index(['date','Asin'])
df_1_index.head()

Empty DataFrame
Columns: []
Index: [(2020-12-31 00:00:00, B07TM3LRVB), (2020-12-31 00:00:00, B0849NLNTQ), (2020-12-31 00:00:00, B08332221J), (2020-12-31 00:00:00, B07MK32L49), (2020-12-31 00:00:00, B08976V1BZ)]

In [ ]:
df_2_index = df_2.set_index(['date','Asin'])
df_2_index

Quantity  Invoice Amount  Item Promo Discount
date       Asin                                                     
2020-12-31 B07TM3LRVB       1.0          1135.0                  0.0
2021-01-01 B07TM3LRVB       1.0          1135.0                  0.0
           B08332221J       1.0          1499.0                  0.0
           B07TM3LRVB       1.0          1135.0                  0.0
           B07TM3LRVB       1.0          1135.0                  0.0

In [ ]:
df_merge = df_2_index.merge(df_1_index,how='outer',on=['date','Asin'])
df_merge

Quantity  Invoice Amount  Item Promo Discount
date       Asin                                                     
2020-12-31 B07TM3LRVB       1.0          1135.0                  0.0
2021-01-01 B07TM3LRVB       1.0          1135.0                  0.0
           B07TM3LRVB       1.0          1135.0                  0.0
           B07TM3LRVB       1.0          1135.0                  0.0
           B08332221J       1.0          1499.0                  0.0
2020-12-31 B0849NLNTQ       NaN             NaN                  NaN
           B08332221J       NaN             NaN                  NaN
           B07MK32L49       NaN             NaN                  NaN
           B08976V1BZ       NaN             NaN                  NaN

In [ ]:
df_2.merge(df_1,how ='outer',on=['date','Asin'])

,Quantity,Asin,Invoice Amount,Item Promo Discount,date
0,1.0,B07TM3LRVB,1135.0,0.0,2020-12-31
1,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
2,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
3,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
4,1.0,B08332221J,1499.0,0.0,2021-01-01
5,NaN,B0849NLNTQ,NaN,NaN,2020-12-31
6,NaN,B08332221J,NaN,NaN,2020-12-31
7,NaN,B07MK32L49,NaN,NaN,2020-12-31
8,NaN,B08976V1BZ,NaN,NaN,2020-12-31


In [ ]:
df_merge.reset_index(inplace = True)

In [ ]:
df_merge

,date,Asin,Quantity,Invoice Amount,Item Promo Discount
0,2020-12-31,B07TM3LRVB,1.0,1135.0,0.0
1,2021-01-01,B07TM3LRVB,1.0,1135.0,0.0
2,2021-01-01,B07TM3LRVB,1.0,1135.0,0.0
3,2021-01-01,B07TM3LRVB,1.0,1135.0,0.0
4,2021-01-01,B08332221J,1.0,1499.0,0.0
5,2020-12-31,B0849NLNTQ,NaN,NaN,NaN
6,2020-12-31,B08332221J,NaN,NaN,NaN
7,2020-12-31,B07MK32L49,NaN,NaN,NaN
8,2020-12-31,B08976V1BZ,NaN,NaN,NaN


In [ ]:
df_merge['Quantity']= df_merge['Quantity'].fillna(0)

In [ ]:
df_merge

,date,Asin,Quantity,Invoice Amount,Item Promo Discount
0,2020-12-31,B07TM3LRVB,1.0,1135.0,0.0
1,2021-01-01,B07TM3LRVB,1.0,1135.0,0.0
2,2021-01-01,B07TM3LRVB,1.0,1135.0,0.0
3,2021-01-01,B07TM3LRVB,1.0,1135.0,0.0
4,2021-01-01,B08332221J,1.0,1499.0,0.0
5,2020-12-31,B0849NLNTQ,0.0,NaN,NaN
6,2020-12-31,B08332221J,0.0,NaN,NaN
7,2020-12-31,B07MK32L49,0.0,NaN,NaN
8,2020-12-31,B08976V1BZ,0.0,NaN,NaN


FInally after hours of deliberation we have done it. We have created what we wanted, now we have details of all the asins each day. Whether in a particular day a sale has been made or not.  

Now we will do this for 1 month, and from there we will getr



**I think we can add one more column that is one is total sum of the product for each day and second is product cost. We can do that maybe in the next part, we can also use np.where condition to create a column with prices for one product for example, if quantity is 1, then price is invoice amount otherwise it is price/quanity.**


For now why we are not adding it is that i think the alogo can find its meaning of product price from this data itself, say quanity 5 and total value is 2000. 

#### We are trying for 1 month

In [ ]:
df_new = df_dicts[2]
df_new_1 = df_new.copy()

In [ ]:
df_new_1['date'] = df_new['Order Date'].dt.date

In [ ]:
df_new  = df_new_1.drop(['Order Date'],axis=1)

In [ ]:
asin_list = df_new.Asin.unique()
asin_list

array(['B07TM3LRVB', 'B0849NLNTQ', 'B08332221J', 'B07MK32L49', 'B08976V1BZ', 'B0859193NJ', 'B07TJ1HDTM', 'B0858X26QM', 'B07KYFHTGF', 'B0859B5J8T', 'B0897LPBHR', 'B084JJWJ31', 'B07W1YYQ9G',
       'B07ZBCTB9N', 'B07S8NC532', 'B07YC69QKT', 'B07Z7QX6ST'], dtype=object)

In [ ]:
df_new['date'] = pd.to_datetime(df_new['date'])

In [ ]:
# Removing refund and return
cond = df_new['Invoice Amount'] > 0

df_new_2  = df_new.where(cond)
df_new_2.dropna(how='all',inplace = True)

In [ ]:
max_date = max(df_new_2['date']) 
min_date = (min(df_new_2['date']))

In [ ]:
# finding all the dates btw max and min

date_list  = pd.date_range(min_date,max_date)

In [ ]:
df_asin = pd.DataFrame(columns = ['date', 'Asin'])

In [ ]:
index = 0
while True:

  for date in (date_list) :
    for asin in (asin_list):
      df_asin.loc[index] = [date,asin]
      index = index + 1

  break    

In [ ]:
df_asin.shape

(544, 2)

In [ ]:
df_asin.head()

,date,Asin
0,2020-12-31,B07TM3LRVB
1,2020-12-31,B0849NLNTQ
2,2020-12-31,B08332221J
3,2020-12-31,B07MK32L49
4,2020-12-31,B08976V1BZ


In [ ]:
df_new = df_new_2

In [ ]:
df_new.dtypes

Quantity                      float64
Asin                           object
Invoice Amount                float64
Item Promo Discount           float64
date                   datetime64[ns]
dtype: object

In [ ]:
df_asin.dtypes

date    datetime64[ns]
Asin            object
dtype: object

In [ ]:
df_merge = df_new.merge(df_asin, how='outer', on=['date','Asin'])
df_merge

,Quantity,Asin,Invoice Amount,Item Promo Discount,date
0,1.0,B07TM3LRVB,1135.0,0.0,2020-12-31
1,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
2,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
3,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
4,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
...,...,...,...,...,...
810,NaN,B084JJWJ31,NaN,NaN,2021-01-28
811,NaN,B07KYFHTGF,NaN,NaN,2021-01-29
812,NaN,B08976V1BZ,NaN,NaN,2021-01-29
813,NaN,B08332221J,NaN,NaN,2021-01-29


In [ ]:
df_merge = df_new.merge(df_asin, how='outer', on=['date','Asin'])
df_merge

,Quantity,Asin,Invoice Amount,Item Promo Discount,date
0,1.0,B07TM3LRVB,1135.0,0.0,2020-12-31
1,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
2,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
3,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
4,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01
...,...,...,...,...,...
1101,NaN,B07W1YYQ9G,NaN,NaN,2021-01-31
1102,NaN,B07ZBCTB9N,NaN,NaN,2021-01-31
1103,NaN,B07S8NC532,NaN,NaN,2021-01-31
1104,NaN,B07YC69QKT,NaN,NaN,2021-01-31


In [ ]:
df_merge['price'] = df_merge['Invoice Amount']/df_merge['Quantity']

In [ ]:
df_merge['Quantity'] = df_merge['Quantity'].fillna(0)

In [ ]:
df_merge['Invoice Amount'] = df_merge['Invoice Amount'].fillna(0)

In [ ]:
df_merge['Item Promo Discount'] = df_merge['Item Promo Discount'].fillna(0)

In [ ]:
df_merge.head()

,Quantity,Asin,Invoice Amount,Item Promo Discount,date,price
0,1.0,B07TM3LRVB,1135.0,0.0,2020-12-31,1135.0
1,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01,1135.0
2,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01,1135.0
3,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01,1135.0
4,1.0,B07TM3LRVB,1135.0,0.0,2021-01-01,1135.0


In [ ]:
df_price = df_merge.groupby('Asin')['price'].agg(lambda x: np.mean(pd.Series.mode(x))).reset_index()

In [ ]:
df_merge.nunique()

Quantity                5
Asin                   17
Invoice Amount         41
Item Promo Discount    14
date                   32
price                  31
dtype: int64

In [ ]:
df = df_merge.set_index('Asin')

In [ ]:
df_price = df_price.set_index('Asin')

In [ ]:
df.update(df_price,overwrite = False, join = 'left')

In [ ]:
# df = df_merge.update(df_price, overwrite = False, join = 'left')

In [ ]:
df.reset_index(inplace = True)

In [ ]:
x = df.groupby(['date','Asin','Item Promo Discount','price']).sum()
x

Quantity  Invoice Amount
date       Asin       Item Promo Discount price                             
2020-12-31 B07KYFHTGF 0.0                 1995.000       0.0             0.0
           B07MK32L49 0.0                 149.000        0.0             0.0
           B07S8NC532 0.0                 2099.000       0.0             0.0
           B07TJ1HDTM 0.0                 1649.000       0.0             0.0
           B07TM3LRVB 0.0                 1135.000       1.0          1135.0
...                                                      ...             ...
2021-01-31 B0858X26QM 0.0                 599.000        0.0             0.0
           B0859193NJ 0.0                 599.000        0.0             0.0
           B0859B5J8T 0.0                 499.000        1.0           499.0
           B08976V1BZ 0.0                 1599.000       0.0             0.0
           B0897LPBHR 0.0                 1896.375       0.0             0.0

[618 rows x 2 columns]

In [ ]:
x.reset_index()

,date,Asin,Item Promo Discount,price,Quantity,Invoice Amount
0,2020-12-31,B07KYFHTGF,0.0,1995.000,0.0,0.0
1,2020-12-31,B07MK32L49,0.0,149.000,0.0,0.0
2,2020-12-31,B07S8NC532,0.0,2099.000,0.0,0.0
3,2020-12-31,B07TJ1HDTM,0.0,1649.000,0.0,0.0
4,2020-12-31,B07TM3LRVB,0.0,1135.000,1.0,1135.0
...,...,...,...,...,...,...
613,2021-01-31,B0858X26QM,0.0,599.000,0.0,0.0
614,2021-01-31,B0859193NJ,0.0,599.000,0.0,0.0
615,2021-01-31,B0859B5J8T,0.0,499.000,1.0,499.0
616,2021-01-31,B08976V1BZ,0.0,1599.000,0.0,0.0


In [ ]:
x.loc['2021-01-31']

Quantity  Invoice Amount
date       Asin       Item Promo Discount price                             
2021-01-31 B07KYFHTGF  0.00               1995.000       0.0            0.00
           B07MK32L49  0.00               149.000        0.0            0.00
           B07S8NC532  0.00               2099.000       0.0            0.00
           B07TJ1HDTM  0.00               1649.000       0.0            0.00
           B07TM3LRVB -56.75              1078.250       6.0         6469.50
                       0.00               1135.000      12.0        13620.00
           B07W1YYQ9G  0.00               399.000        0.0            0.00
           B07YC69QKT  0.00               1649.000       0.0            0.00
           B07Z7QX6ST  0.00               486.525        0.0            0.00
           B07ZBCTB9N  0.00               1135.000       0.0            0.00
           B08332221J -74.95              1424.050       1.0         1424.05
           B0849NLNTQ  0.00               1599.000       2.0         3198.00
           B084JJWJ31  0.00               599.000        0.0            0.00
           B0858X26QM  0.00               599.000        0.0            0.00
           B0859193NJ  0.00               599.000        0.0            0.00
           B0859B5J8T  0.00               499.000        1.0          499.00
           B08976V1BZ  0.00               1599.000       0.0            0.00
           B0897LPBHR  0.00               1896.375       0.0            0.00

Working with all the dataframes

In [ ]:
# concat all df
def process_data(df_dicts):

  df_main = pd.DataFrame()


  for i in range(len(df_dicts)):
  # we need to convert the Order date column into date_time
  # if df_dicts[i]['Order Date'].dtype != 'datetime64[ns]' :
    df_copy = df_dicts[i].copy()

    df_copy['Order Date'] = pd.to_datetime(df_dicts[i]['Order Date'])

    df_copy['date'] = df_copy['Order Date'].dt.date #

    df_dicts[i] = df_copy.drop(['Order Date'],axis=1) 


    asin_list = df_dicts[i].Asin.unique()

    # Removing refund and return
    cond = df_dicts[i]['Invoice Amount'] > 0

    df_copy_2  = df_dicts[i].where(cond)
    df_copy_2.dropna(how='all',inplace = True)

    max_date = max(df_copy_2['date']) 
    min_date = (min(df_copy_2['date']))
    date_list  = pd.date_range(min_date,max_date)

    df_asin = pd.DataFrame(columns = ['date', 'Asin'])
    index = 0
    while True:

      for date in (date_list) :
        for asin in (asin_list):
          df_asin.loc[index] = [date,asin]
          index = index + 1

      break    

    df_dicts[i] = df_copy_2

    df_merge = df_copy_2.merge(df_asin, how='outer', on=['date','Asin'])
    
    df_merge['price'] = df_merge['Invoice Amount']/df_merge['Quantity']
    df_merge['Quantity'] = df_merge['Quantity'].fillna(0)
    df_merge['Invoice Amount'] = df_merge['Invoice Amount'].fillna(0)
    df_merge['Item Promo Discount'] = df_merge['Item Promo Discount'].fillna(0)

    df_price = df_merge.groupby('Asin')['price'].agg(lambda x: np.mean(pd.Series.mode(x))).reset_index()

    df = df_merge.set_index('Asin')
    df_price = df_price.set_index('Asin')

    df.update(df_price,overwrite = False, join = 'left')

    df = df.groupby(['date','Asin','Item Promo Discount','price']).sum()
    df.reset_index(inplace=True)

    df_main = pd.concat([df_main , df])

  
  return df_main    


In [ ]:
  df_main = pd.DataFrame()


#for i in range(len(df_dicts)):

df_copy = df_dicts[2].copy()

df_copy['Order Date'] = pd.to_datetime(df_dicts[2]['Order Date'])

df_copy['date'] = df_copy['Order Date'].dt.date #

df_dicts[2] = df_copy.drop(['Order Date'],axis=1) 


asin_list = df_dicts[2].Asin.unique()

# Removing refund and return
cond = df_dicts[2]['Invoice Amount'] > 0

df_copy_2  = df_dicts[2].where(cond)
df_copy_2.dropna(how='all',inplace = True)

max_date = max(df_copy_2['date']) 
min_date = (min(df_copy_2['date']))
date_list  = pd.date_range(min_date,max_date)

df_asin = pd.DataFrame(columns = ['date', 'Asin'])
index = 0

while True:

  for date in (date_list) :
    for asin in (asin_list):
      df_asin.loc[index] = [date,asin]
      index = index + 1

  break    

df_dicts[2] = df_copy_2

df_merge = df_copy_2.merge(df_asin, how='outer', on=['date','Asin'])

df_merge['price'] = df_merge['Invoice Amount']/df_merge['Quantity']
df_merge['Quantity'] = df_merge['Quantity'].fillna(0)
df_merge['Invoice Amount'] = df_merge['Invoice Amount'].fillna(0)
df_merge['Item Promo Discount'] = df_merge['Item Promo Discount'].fillna(0)

df_price = df_merge.groupby('Asin')['price'].agg(lambda x: np.mean(pd.Series.mode(x))).reset_index()

df = df_merge.set_index('Asin')
df_price = df_price.set_index('Asin')

df.update(df_price,overwrite = False, join = 'left')

df = df.groupby(['date','Asin','Item Promo Discount','price']).sum()
df.reset_index(inplace=True)

df_main = pd.concat([df_main , df])

  

IndentationError: ignored

In [ ]:
process_data(df_dicts)

ValueError: ignored

In [ ]:
df_new = df_new_2

In [ ]:
df_merge = df_new.merge(df_asin, how='outer', on=['date','Asin'])
df_merge

In [ ]:
df_merge['price'] = df_merge['Invoice Amount']/df_merge['Quantity']

In [ ]:
df_merge['Quantity'] = df_merge['Quantity'].fillna(0)

In [ ]:
df_merge['Invoice Amount'] = df_merge['Invoice Amount'].fillna(0)

In [ ]:
df_merge['Item Promo Discount'] = df_merge['Item Promo Discount'].fillna(0)

In [ ]:
df_price = df_merge.groupby('Asin')['price'].agg(lambda x: np.mean(pd.Series.mode(x))).reset_index()

In [ ]:
df = df_merge.set_index('Asin')

In [ ]:
df_price = df_price.set_index('Asin')

In [ ]:
df.update(df_price,overwrite = False, join = 'left')

In [ ]:
x = df.groupby(['date','Asin','Item Promo Discount','price']).sum()
x

Quantity  Invoice Amount
date       Asin       Item Promo Discount price                             
2020-12-31 B07KYFHTGF 0.0                 1995.000       0.0             0.0
           B07MK32L49 0.0                 149.000        0.0             0.0
           B07S8NC532 0.0                 2099.000       0.0             0.0
           B07TJ1HDTM 0.0                 1649.000       0.0             0.0
           B07TM3LRVB 0.0                 1135.000       1.0          1135.0
...                                                      ...             ...
2021-01-31 B0858X26QM 0.0                 599.000        0.0             0.0
           B0859193NJ 0.0                 599.000        0.0             0.0
           B0859B5J8T 0.0                 499.000        1.0           499.0
           B08976V1BZ 0.0                 1599.000       0.0             0.0
           B0897LPBHR 0.0                 1896.375       0.0             0.0

[618 rows x 2 columns]

In [ ]:
df_new = df_dicts[2]
df_new_1 = df_new.copy()

In [ ]:
df_new_1['date'] = df_new['Order Date'].dt.date

In [ ]:
df_new  = df_new_1.drop(['Order Date'],axis=1)

In [ ]:
asin_list = df_new.Asin.unique()
asin_list

array(['B07TM3LRVB', 'B0849NLNTQ', 'B08332221J', 'B07MK32L49', 'B08976V1BZ', 'B0859193NJ', 'B07TJ1HDTM', 'B0858X26QM', 'B07KYFHTGF', 'B0859B5J8T', 'B0897LPBHR', 'B084JJWJ31', 'B07W1YYQ9G',
       'B07ZBCTB9N', 'B07S8NC532', 'B07YC69QKT', 'B07Z7QX6ST'], dtype=object)

In [ ]:
df_new['date'] = pd.to_datetime(df_new['date'])

In [ ]:
# Removing refund and return
cond = df_new['Invoice Amount'] > 0

df_new_2  = df_new.where(cond)
df_new_2.dropna(how='all',inplace = True)

In [ ]:
max_date = max(df_new_2['date']) 
min_date = (min(df_new_2['date']))

In [ ]:
# finding all the dates btw max and min

date_list  = pd.date_range(min_date,max_date)

In [ ]:
df_asin = pd.DataFrame(columns = ['date', 'Asin'])